# Airbyte Shopify (已弃用)

注意：此连接器专用加载器已弃用。请改用 [`AirbyteLoader`](/docs/integrations/document_loaders/airbyte)。

>[Airbyte](https://github.com/airbytehq/airbyte) 是一个用于 ELT 管道的数据集成平台，可将数据从 API、数据库和文件提取到仓库和数据湖。它拥有最大的到数据仓库和数据库的 ELT 连接器目录。

此加载器将 Shopify 连接器公开为文档加载器，允许您将各种 Shopify 对象加载为文档。

## 安装

首先，您需要安装 `airbyte-source-shopify` Python 包。

In [ ]:
%pip install --upgrade --quiet  airbyte-source-shopify

## 示例

请参阅 [Airbyte 文档页面](https://docs.airbyte.com/integrations/sources/shopify/) 以了解如何配置读取器。

配置对象应遵循的 JSON schema 可在 Github 上找到：[https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-shopify/source_shopify/spec.json](https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-shopify/source_shopify/spec.json)。

大致结构如下：
```python
{
    "start_date": "<开始检索记录的日期，ISO 格式，例如 2020-10-20T00:00:00Z>",
    "shop": "<您想从中检索文档的商店名称>",
    "credentials": {
        "auth_method": "api_password",
        "api_password": "<您的 API 密码>"
    }
}
```

默认情况下，所有字段都将作为文档中的元数据存储，文本内容将设置为空字符串。通过转换读取器返回的文档来构建文档的文本内容。

In [ ]:
from langchain_community.document_loaders.airbyte import AirbyteShopifyLoader

config = {
    # your shopify configuration
}

loader = AirbyteShopifyLoader(
    config=config, stream_name="orders"
)  # check the documentation linked above for a list of all streams

现在您可以通过常规方式加载文档

In [ ]:
docs = loader.load()

作为 `load` 返回一个列表，它将一直阻塞直到所有文档加载完毕。为了更好地控制此过程，您还可以使用 `lazy_load` 方法，该方法返回一个迭代器：

In [ ]:
docs_iterator = loader.lazy_load()

请记住，默认情况下，页面内容为空，而 metadata 对象包含记录中的所有信息。要以不同的方式创建文档，请在创建加载器时传入一个 `record_handler` 函数：

In [ ]:
from langchain_core.documents import Document


def handle_record(record, id):
    return Document(page_content=record.data["title"], metadata=record.data)


loader = AirbyteShopifyLoader(
    config=config, record_handler=handle_record, stream_name="orders"
)
docs = loader.load()

## 增量加载

某些流支持增量加载，这意味着源会跟踪已同步的记录，并且不会再次加载它们。这对于数据量大且经常更新的源非常有用。

要利用此功能，请存储加载器的 `last_state` 属性，并在再次创建加载器时将其传入。这将确保只加载新记录。

In [ ]:
last_state = loader.last_state  # store safely

incremental_loader = AirbyteShopifyLoader(
    config=config, stream_name="orders", state=last_state
)

new_docs = incremental_loader.load()